In [12]:
from ccnet_spark.pipe_line import Pipeline, Config
import time
from pyspark.sql import SparkSession
import sys

pipeline=[
    "real_len",
    "hash",
    "dedup_keep",
    "lid",
    "sp",
    "lm",
    "pp_bucket",
    "drop",
]

spark = (
    SparkSession.builder.appName("CCNETSpark_ONLY")
    # .master("local[*]")
    .config("spark.executor.memory", "100g")
    .config("spark.driver.memory", "100g")
    .config("spark.driver.maxResultSize", "100g")
    .config("spark.sql.execution.arrow.pyspark.enabled", "true")
    # .config("spark.executor.extraJavaOptions", "-XX:+PrintGCDetails -XX:+PrintGCDateStamps -XX:+PrintGCTimeStamps")
    # .config("spark.driver.extraJavaOptions", "-XX:+PrintGCDetails -XX:+PrintGCDateStamps -XX:+PrintGCTimeStamps")
    .getOrCreate()
)
config = Config(
        isSample=True,
        n_segments=1,
        sampleRate=0.01,
        cache_dir="../../cached_data/",
        output_dir="../../cached_data/",
        fasttext_model_path="../../cc_net/bin/lid.bin",
        lm_dir="../../cc_net/data/lm_sp",
        cutoff_csv_path="../../cc_net/cc_net/data/cutoff.csv",
        dump="2019-18",
        pipeline=pipeline,
    )
pipeline = Pipeline(config,spark)
df = pipeline.load_data()

24/04/11 09:34:31 WARN DataSource: All paths were ignored:
  file:/root/wxl_folder/cached_data/sdf_parquet/2019-18/_sampleRate_1_segment_0_min_len_300.parquet
2024-04-11 09:34 INFO 1860083:root - load segment 0, with sampleRate:1.0%,min_len:300,with date:2019-18


In [13]:
print(df.columns)
df.select("url","length","nlines","raw_content","cc_segment").show(5)

['url', 'date_download', 'digest', 'length', 'nlines', 'source_domain', 'title', 'raw_content', 'cc_segment']
+--------------------+------+------+--------------------------------+--------------------+
|                 url|length|nlines|                     raw_content|          cc_segment|
+--------------------+------+------+--------------------------------+--------------------+
|http://rgdv.com.c...|  2286|    63|  更多\n更多\n全站搜索\n更多\...|CC-MAIN-201904181...|
|https://www.place...|  4964|   141|            Explore\nLife\nFo...|CC-MAIN-201904181...|
|http://voidslave....|369505|  7353|            sample lunch and ...|CC-MAIN-201904181...|
|https://inaba-afi...| 14478|   260|年中行事\nこどもの日\n母の日\...|CC-MAIN-201904181...|
|https://www.wamp....|  1694|    41|            Skip to main cont...|CC-MAIN-201904181...|
+--------------------+------+------+--------------------------------+--------------------+
only showing top 5 rows



In [14]:
pipeline.run_step("real_len")
print(pipeline.df.columns)
pipeline.df.select("url","length","nlines","raw_content","cc_segment").show(5)

['url', 'date_download', 'digest', 'length', 'nlines', 'source_domain', 'title', 'raw_content', 'cc_segment']
+--------------------+------+------+--------------------------------+--------------------+
|                 url|length|nlines|                     raw_content|          cc_segment|
+--------------------+------+------+--------------------------------+--------------------+
|http://rgdv.com.c...|  1054|    63|  更多\n更多\n全站搜索\n更多\...|CC-MAIN-201904181...|
|https://www.place...|  4898|   141|            Explore\nLife\nFo...|CC-MAIN-201904181...|
|http://voidslave....|369489|  7353|            sample lunch and ...|CC-MAIN-201904181...|
|https://inaba-afi...|  5299|   260|年中行事\nこどもの日\n母の日\...|CC-MAIN-201904181...|
|https://www.wamp....|  1672|    41|            Skip to main cont...|CC-MAIN-201904181...|
+--------------------+------+------+--------------------------------+--------------------+
only showing top 5 rows



In [15]:
pipeline.run_step("hash")
print(pipeline.df.columns)
pipeline.df.select("url","length","nlines","raw_content","cc_segment",'exploded_content', 'hash_value').show(5)

['url', 'date_download', 'digest', 'length', 'nlines', 'source_domain', 'title', 'raw_content', 'cc_segment', 'exploded_content', 'hash_value']
+--------------------+------+------+------------------------------+--------------------+----------------+--------------------+
|                 url|length|nlines|                   raw_content|          cc_segment|exploded_content|          hash_value|
+--------------------+------+------+------------------------------+--------------------+----------------+--------------------+
|http://rgdv.com.c...|  1054|    63|更多\n更多\n全站搜索\n更多\...|CC-MAIN-201904181...|       {0, 更多}|[9B 0C 6C 78 58 B...|
|http://rgdv.com.c...|  1054|    63|更多\n更多\n全站搜索\n更多\...|CC-MAIN-201904181...|       {1, 更多}|[9B 0C 6C 78 58 B...|
|http://rgdv.com.c...|  1054|    63|更多\n更多\n全站搜索\n更多\...|CC-MAIN-201904181...|   {2, 全站搜索}|[97 B8 03 99 98 C...|
|http://rgdv.com.c...|  1054|    63|更多\n更多\n全站搜索\n更多\...|CC-MAIN-201904181...|       {3, 更多}|[9B 0C 6C 78 58 B...|
|http://rgdv.com.

In [16]:
pipeline.run_step("dedup_keep")
print(pipeline.df.columns)
pipeline.df.select("url","length","nlines","raw_content","cc_segment",'raw_line_id','original_length','original_nlines').show(5)

['digest', 'url', 'date_download', 'source_domain', 'cc_segment', 'original_length', 'original_nlines', 'title', 'nlines', 'raw_content', 'raw_line_id', 'length']


+--------------------+------+------+--------------------+--------------------+--------------------+---------------+---------------+
|                 url|length|nlines|         raw_content|          cc_segment|         raw_line_id|original_length|original_nlines|
+--------------------+------+------+--------------------+--------------------+--------------------+---------------+---------------+
|http://ecrack1.ui...| 42384|    64|Ring ring\n1234 6...|CC-MAIN-201904181...|[0, 1, 2, 3, 4, 5...|          44461|            108|
|https://romankhon...|  2943|   124|رفتن به مطلب\nفرو...|CC-MAIN-201904181...|[0, 1, 2, 3, 4, 5...|           3655|            189|
|https://www.obesi...|  1597|    71|Navigation\nBaria...|CC-MAIN-201904181...|[0, 1, 2, 3, 4, 5...|           3519|            185|
|http://publicrepo...|  7398|    72|Reports on Long-T...|CC-MAIN-201904181...|[0, 4, 6, 7, 8, 9...|           7742|             96|
|http://www.e-isla...|  4531|    38|e-Ислам\nНовостно...|CC-MAIN-201904181..

In [20]:
pipeline.run_step("lid")
print(pipeline.df.columns)
pipeline.df.select("url","length","nlines","raw_content",'raw_line_id','original_length','original_nlines','lang','score').show(5)

['digest', 'url', 'date_download', 'source_domain', 'cc_segment', 'original_length', 'original_nlines', 'title', 'nlines', 'raw_content', 'raw_line_id', 'length', 'lang', 'score']
+--------------------+------+------+--------------------+--------------------+---------------+---------------+----+-----+
|                 url|length|nlines|         raw_content|         raw_line_id|original_length|original_nlines|lang|score|
+--------------------+------+------+--------------------+--------------------+---------------+---------------+----+-----+
|http://ecrack1.ui...| 42384|    64|Ring ring\n1234 6...|[0, 1, 2, 3, 4, 5...|          44461|            108|null| null|
|https://romankhon...|  2943|   124|رفتن به مطلب\nفرو...|[0, 1, 2, 3, 4, 5...|           3655|            189|  fa| 0.99|
|https://www.obesi...|  1597|    71|Navigation\nBaria...|[0, 1, 2, 3, 4, 5...|           3519|            185|  en|  0.7|
|http://publicrepo...|  7398|    72|Reports on Long-T...|[0, 4, 6, 7, 8, 9...|          

In [22]:
pipeline.run_step("sp")
print(pipeline.df.columns)
pipeline.df.select("url","length","nlines","raw_content",'raw_line_id','lang','score','tokenized').show(5)

['digest', 'url', 'date_download', 'source_domain', 'cc_segment', 'original_length', 'original_nlines', 'title', 'nlines', 'raw_content', 'raw_line_id', 'length', 'lang', 'score', 'tokenized']


+--------------------+------+------+--------------------+--------------------+----+-----+--------------------+
|                 url|length|nlines|         raw_content|         raw_line_id|lang|score|           tokenized|
+--------------------+------+------+--------------------+--------------------+----+-----+--------------------+
|http://ecrack1.ui...| 42384|    64|Ring ring\n1234 6...|[0, 1, 2, 3, 4, 5...|null| null|                null|
|https://romankhon...|  2943|   124|رفتن به مطلب\nفرو...|[0, 1, 2, 3, 4, 5...|  fa| 0.99|▁رفتن ▁به ▁مطلب ف...|
|https://www.obesi...|  1597|    71|Navigation\nBaria...|[0, 1, 2, 3, 4, 5...|  en|  0.7|▁navigation bari ...|
|http://publicrepo...|  7398|    72|Reports on Long-T...|[0, 4, 6, 7, 8, 9...|  en| 0.93|▁reports ▁on ▁lon...|
|http://www.e-isla...|  4531|    38|e-Ислам\nНовостно...|[0, 1, 2, 3, 5, 6...|  ru| 0.99|▁e - ислам ново с...|
+--------------------+------+------+--------------------+--------------------+----+-----+--------------------+
o

In [25]:
pipeline.run_step("lm")
print(pipeline.df.columns)
pipeline.df.select("url","length","nlines","raw_content",'raw_line_id','lang','score','perplexity').show(5)

['digest', 'url', 'date_download', 'source_domain', 'cc_segment', 'original_length', 'original_nlines', 'title', 'nlines', 'raw_content', 'raw_line_id', 'length', 'lang', 'score', 'tokenized', 'perplexity']


+--------------------+------+------+--------------------+--------------------+----+-----+----------+
|                 url|length|nlines|         raw_content|         raw_line_id|lang|score|perplexity|
+--------------------+------+------+--------------------+--------------------+----+-----+----------+
|http://ecrack1.ui...| 42384|    64|Ring ring\n1234 6...|[0, 1, 2, 3, 4, 5...|null| null|      null|
|https://romankhon...|  2943|   124|رفتن به مطلب\nفرو...|[0, 1, 2, 3, 4, 5...|  fa| 0.99|    2919.3|
|https://www.obesi...|  1597|    71|Navigation\nBaria...|[0, 1, 2, 3, 4, 5...|  en|  0.7|    4605.3|
|http://publicrepo...|  7398|    72|Reports on Long-T...|[0, 4, 6, 7, 8, 9...|  en| 0.93|     313.1|
|http://www.e-isla...|  4531|    38|e-Ислам\nНовостно...|[0, 1, 2, 3, 5, 6...|  ru| 0.99|     166.6|
+--------------------+------+------+--------------------+--------------------+----+-----+----------+
only showing top 5 rows



In [28]:
pipeline.run_step("pp_bucket")
print(pipeline.df.columns)
pipeline.df.select("url","length","nlines","raw_content",'lang','score','perplexity','bucket').show(5)

['digest', 'url', 'date_download', 'source_domain', 'cc_segment', 'original_length', 'original_nlines', 'title', 'nlines', 'raw_content', 'raw_line_id', 'length', 'lang', 'score', 'tokenized', 'perplexity', 'bucket']


+--------------------+------+------+--------------------+----+-----+----------+------+
|                 url|length|nlines|         raw_content|lang|score|perplexity|bucket|
+--------------------+------+------+--------------------+----+-----+----------+------+
|http://ecrack1.ui...| 42384|    64|Ring ring\n1234 6...|null| null|      null|   all|
|https://romankhon...|  2943|   124|رفتن به مطلب\nفرو...|  fa| 0.99|    2919.3|  tail|
|https://www.obesi...|  1597|    71|Navigation\nBaria...|  en|  0.7|    4605.3|  tail|
|http://publicrepo...|  7398|    72|Reports on Long-T...|  en| 0.93|     313.1|  head|
|http://www.e-isla...|  4531|    38|e-Ислам\nНовостно...|  ru| 0.99|     166.6|  head|
+--------------------+------+------+--------------------+----+-----+----------+------+
only showing top 5 rows



In [29]:
pipeline.run_step("drop")
print(pipeline.df.columns)

['digest', 'url', 'date_download', 'source_domain', 'cc_segment', 'original_length', 'original_nlines', 'title', 'nlines', 'raw_content', 'raw_line_id', 'length', 'lang', 'score', 'perplexity', 'bucket']
